In [ ]:
import sys

def print_debug(*p, **kw):
    kw["file"] = sys.stderr
    print("DEBUG " if "sep" in kw and kw["sep"]=="" else "DEBUG", *p, **kw)

def print_exception(*p, **kw):
    kw["file"] = sys.stderr
    print("EXCEPTION " if "sep" in kw and kw["sep"]=="" else "EXCEPTION", *p, **kw)

def print_testing(*p, **kw):
    kw["file"] = sys.stderr
    bp = "TESTING"
    print(bp+" " if "sep" in kw and kw["sep"]=="" else bp, *p, **kw)


In [ ]:
import sys

class IllegalCellValue(Exception):
    pass

    def __init__(self, msg=None):
        Exception.__init__(self)
        if msg is not None:
            print_exception(msg)

def coord_to_square_coord(coord):
    return ((coord[0] // 3) * 3, (coord[1] // 3) * 3)


In [ ]:
class Cell(object):
    VALID_CELL_VALUES = {1, 2, 3, 4, 5, 6, 7, 8, 9}
    
    def __init__(self, r, c):
        self._r = r
        self._c = c
        self._id = "cell_%s" % (tuple([self._r, self._c]),)
        self._parents = {}
        self._value = None

    def add_parent(self, parent, parent_type):
        # if (self._r, self._c) == (4, 7):
        #     print_debug("add_parent to (4,7):", parent, parent_type)
            
        self._parents[parent_type] = parent

    def set_value(self, v):
        if v not in self.VALID_CELL_VALUES:
            raise IllegalCellValue
            
        if self._value is not None:
            raise IllegalCellValue
            
        # test every parent, then do it for real
        ordered_parents = [p for p in self._parents.values() if isinstance(p, Threes)]
        ordered_parents.extend([p for p in self._parents.values() if isinstance(p, Nines)])
        
        # print_debug("unordered parents of", self._id, "are", self._parents.values())
        # print_debug("ordered parents of", self._id, "are", ordered_parents)
        
        for testing in [True, False]:
            for p in ordered_parents:
                p.add_value(v, testing=testing)
        
        self._value = v
        
    def clear_value(self):
        if self._value is None:
            raise IllegalCellValue(msg="%s does not contain a value" % (self._id))
        
        for k, p in self._parents.items():
            p.delete_value(self._value)

        self._value = None
   
    def get_guesses(self):
        guesses = set()
        if self._value is None:
            guesses = Cell.VALID_CELL_VALUES.difference(*[p._child_values for p in self._parents.values()])

        return guesses


In [ ]:
class Threes(object):  # aka segments
    def __init__(self, t, coord):
        self._type = t
        self._coord = coord
        self._id = "%s_%s" % (self._type, self._coord)
        self._cells = dict()
        self._child_values = set()
        self._parents = dict()
        self._siblings = dict()

    def add_cell(self, cell):
        self._cells[cell._id] = cell
        cell.add_parent(self, self._type)
        
    def add_parent(self, parent, parent_type):
        self._parents[parent_type] = parent

    def add_siblings(self, siblings):
        for sibling in siblings:
            self._siblings[sibling._id] = sibling
   
    def add_value(self, v, testing=False):
        if v not in Cell.VALID_CELL_VALUES:
            raise IllegalCellValue
        
        if v in self._child_values:
            raise IllegalCellValue(msg="%s already has a cell with value %s" % (self._id, v))
            
        for sib in self._siblings.values():
            if sib.has_value(v):
                raise IllegalCellValue(msg="%s has a sibling %s that already has a cell with value %s" % (self._id, 
                                                                                                          sib._id,
                                                                                                          v))
            
        if not testing:
            self._child_values.add(v)
    
    def clear_value(self, v, testing=False):
        if v not in Cell.VALID_CELL_VALUES:
            raise IllegalCellValue
        
        if v not in self._child_values:
            raise IllegalCellValue(msg="%s does not contain a cell with value %s" % (self._id, v))
        
        if not testing:
            try:
                self._child_values.remove(v)
            except KeyError:
                raise IllegalCellValue

    def has_value(self, v):
        return v in self._child_values
    
class RowSegment(Threes):
    def __init__(self, coord):
        Threes.__init__(self, "rowsegment", coord)
    pass

class ColumnSegment(Threes):
    def __init__(self, coord):
        Threes.__init__(self, "colsegment", coord)
    pass


In [ ]:
class Nines(object):

    def __init__(self, t, index):
        self._type = t
        self._index = index
        self._id = "%s_%s" % (self._type, self._index)
        self._cells = dict()
        # print("__init__ id: %s" % self._id)
        self._child_values = set()
        self._all_segments = dict()
        
    def add_cell(self, cell):
        self._cells[cell._id] = cell
        cell.add_parent(self, self._type)

    def add_segment(self, segment):
        self._all_segments[segment._id] = segment
        segment.add_parent(self, self._type)

    def add_value(self, v, testing=False):
        if v not in Cell.VALID_CELL_VALUES:
            raise IllegalCellValue
        
        if v in self._child_values:
            raise IllegalCellValue(msg="%s already has a cell with value %s" % (self._id, v))
        if not testing:
            self._child_values.add(v)
    
    def clear_value(self, v, testing=False):
        if v not in Cell.VALID_CELL_VALUES:
            raise IllegalCellValue
        
        if v not in self._child_values:
            raise IllegalCellValue(msg="%s does not contain a cell with value %s" % (self._id, v))
        
        if not testing:
            try:
                self._child_values.remove(v)
            except KeyError:
                raise IllegalCellValue

class Row(Nines):
    def __init__(self, index):
        # print_debug("init Row(%s)" % index)
        Nines.__init__(self, "row", index)
        # print_debug("... after init Nines(...)")

class Column(Nines):
    def __init__(self, index):
        # print("init Column(%s)" % index)
        Nines.__init__(self, "col", index)
        # print("... after init Nines(...)")

class Square(Nines):
    def __init__(self, index):
        # print("init Square(%s)" % (index, ))
        Nines.__init__(self, "square", index)
        # print("... after init Nines(...)")


In [ ]:
class Board(object):
    NUMBER_OF_ROWS = 9
    NUMBER_OF_COLUMNS = NUMBER_OF_ROWS
    ROW_NUMBERS = range(NUMBER_OF_ROWS)
    COLUMN_NUMBERS = range(NUMBER_OF_COLUMNS)
    SQUARE_COORDINATES = [] 
    
    def _init_create_objects(self):
        # create all cells
        self._all_cells = {(r,c):Cell(r,c) for r in self.__class__.ROW_NUMBERS for c in self.__class__.COLUMN_NUMBERS}
        
        # create squares, rows and columns
        self._all_rows = {r:Row(r) for r in self.__class__.ROW_NUMBERS}
        self._all_columns = {c:Column(c) for c in self.__class__.COLUMN_NUMBERS}
        self._all_squares = {r_c:Square(r_c) for r_c in self.__class__.SQUARE_COORDINATES}
        
        # create row and column segments
        self._all_row_segments = {(r, x):RowSegment((r,x),) for r in self.ROW_NUMBERS for x in range(0,9,3)}
        self._all_column_segments = {(x, c):ColumnSegment((x,c),) for c in self.COLUMN_NUMBERS for x in range(0,9,3)}       
        
        # create consolidated dictionaries
        
        self._all_nines = dict([*[(v._id, v) for v in self._all_rows.values()],
                               *[(v._id, v) for v in self._all_columns.values()],
                               *[(v._id, v) for v in self._all_squares.values()]])
        # print_debug("all nines %s" % (self._all_nines), file=sys.stderr)
        
        self._all_segments = dict([*[(v._id, v) for v in self._all_row_segments.values()],
                                   *[(v._id, v) for v in self._all_column_segments.values()]])
        
    def _init_establish_siblings(self):
        # create sibling relationships
        # ... row siblings in rows
        for row in self.ROW_NUMBERS:
            keys = [coord for coord in [(row, c) for c in range(0, 9, 3)]]
            for _ in range(3): # 3 times
                # print_debug("add rowsegment siblings:", keys)
                self._all_row_segments[keys[0]].add_siblings([self._all_row_segments[keys[1]],
                                                          self._all_row_segments[keys[2]]])
                keys.append(keys.pop(0))  # rotate - remove first item then put it on the end
                
        # ... column siblings in columns
        for column in self.COLUMN_NUMBERS:
            keys = [coord for coord in [(r, column) for r in range(0, 9, 3)]]
            for _ in range(3): # 3 times
                # print_debug("add columnsegment siblings:", keys)
                self._all_column_segments[keys[0]].add_siblings([self._all_column_segments[keys[1]],
                                                          self._all_column_segments[keys[2]]])
                keys.append(keys.pop(0))  # rotate - remove first item then put it on the end
                
        # ... row siblings in squares
        for upperleft_row, upperleft_column in self.SQUARE_COORDINATES:
            keys = [(r, upperleft_column) for r in range(upperleft_row, upperleft_row+3)]
            for _ in range(3): # 3 times
                # print_debug("add columnsegment siblings:", keys)
                self._all_row_segments[keys[0]].add_siblings([self._all_row_segments[keys[1]], 
                                                              self._all_row_segments[keys[2]]])
                keys.append(keys.pop(0))  # rotate - remove first item then put it on the end
                
        # ... column siblings in squares
        for upperleft_row, upperleft_column in self.SQUARE_COORDINATES:
            keys = [(upperleft_row, c) for c in range(upperleft_column, upperleft_column+3)]
            for _ in range(3): # 3 times
                # print_debug("add columnsegment siblings:", keys)
                self._all_column_segments[keys[0]].add_siblings([self._all_column_segments[keys[1]], 
                                                                 self._all_column_segments[keys[2]]])
                keys.append(keys.pop(0))  # rotate - remove first item then put it on the end
                
    def _init_add_threes_to_nines(self):
        
        for segment in self._all_segments.values():
            coord = segment._coord
            
            if issubclass(segment.__class__, RowSegment):
                self._all_rows[coord[0]].add_segment(segment)
            elif issubclass(segment.__class__, ColumnSegment):
                self._all_columns[coord[1]].add_segment(segment)
            else:
                raise TypeError
            
            self._all_squares[coord_to_square_coord(coord)].add_segment(segment)
    
    def __init__(self):
        # doesn't work in static code (?)
        self.__class__.SQUARE_COORDINATES = [(r,c) for r in range(0, self.__class__.NUMBER_OF_ROWS, 3) 
                                             for c in range(0, self.__class__.NUMBER_OF_COLUMNS, 3)] 
        
        self._init_create_objects()  # cells, nines and threes
        self._init_establish_siblings()  # link row segments together, likewise column segments
        self._init_add_threes_to_nines()  # link rows to row segments, columns to column segments
                                            # ... squares to row and column segments
        self._init_add_cells()
        
    def _init_add_cells(self):
        # add cells to squares, rows and columns
        for cell in self._all_cells.values():
            r, c = (cell._r, cell._c)
            row = self._all_rows[r].add_cell(cell)
            col = self._all_columns[c].add_cell(cell)
            sr = (r // 3) * 3
            sc = (c // 3) * 3
            square = self._all_squares[(sr, sc)].add_cell(cell)
            
        # add cells to row and column segments
        for cell in self._all_cells.values():
            r, c = (cell._r, cell._c)
            xr = (r // 3) * 3
            xc = (c // 3) * 3

            self._all_segments["rowsegment_%s" % ((r, xc),)].add_cell(cell)
            self._all_segments["colsegment_%s" % ((xr, c),)].add_cell(cell)
            
            # self._all_segments["row_%s" % ((r, xc),)].dump()
            # self._all_segments["col_%s" % ((xr, c),)].dump()
            
    def get_cell(self, r, c):
        return self._all_cells[(r,c)]
    
    def set_cell_value(self, r_c, v):
        self.get_cell(r_c[0], r_c[1]).set_value(v)

    def clear_cell_value(self, r_c):
        self.get_cell(r_c[0], r_c[1]).clear_value()

    def debug_init(self):
        print_debug("%s" % (self.__class__ ), file=sys.stderr) 
        
    def dump_all_nines(self, suppress_empty=True):
        print_debug("%s" % (self.__class__ )) 

        all_nines = list()
        all_nines += self._all_rows.values()
        all_nines += self._all_columns.values()
        all_nines += self._all_squares.values()
        
        for p in all_nines:
            if suppress_empty and len(p._child_values) == 0:
                continue
                
            print_debug("%s has values %s" % (p._id, p._child_values))

    def dump_nine(self, klass, coord):
        print_debug("%s" % (self.__class__ )) 

        if issubclass(klass, Square):
            source = self._all_squares
        elif issubclass(klass, Row):
            source = self._all_rows
        elif issubclass(klass, Column):
            source = self._all_columns
        else:
            raise ClassError
            
        nine = source[coord]
        print_debug("%s has ...\n... children: %s\n... segments: %s" % (nine._id, 
                                                                        nine._cells.keys(), 
                                                                        nine._all_segments.keys()))

    def dump_all_segments(self, suppress_empty=True):
        print_debug("%s" % (self.__class__ )) 

        # print_debug("row segments:", self._all_row_segments)
        # print_debug("column segments:", self._all_column_segments)
        for s in self._all_segments.values():
            print_debug("all segments:", s._id, s._cells, s._parents, s._siblings)

    def dump_cell(self, c, suppress_empty=True):
        print_debug("%s" % (self.__class__ )) 

        print_debug("cell:", c._id, c._parents)


In [ ]:
def testing_set_cell_value():
    print("testing_set_cell_value()")

    b = Board()
    b.debug_init()

    c44 = b.get_cell(4, 4)
    c45 = b.get_cell(4, 5)

    # c44.clear_value()  # should fail
    c44.set_value(5)

    # c45.set_value(5)  # should fail
    c45.set_value(4)

    b.dump_all_nines()


In [ ]:
def testing_board_set_cell_value():
    print_testing("board_set_cell_value()")

    board = Board()
    # board.debug_init()

    board.set_cell_value((4, 4), 5)
    board.set_cell_value((4, 5), 4)

    board.dump_all_nines()
    # board.dump_all_nines(suppress_empty=False)


In [ ]:
def testing_board_set_cell_value2():
    print_testing("board_set_cell_value2()")

    def set(bored, lyst, suite_msg="???"):
        print_testing("--- suite: %s ---" % (suite_msg))
        for coord, v in lyst:
            print_testing("set cell %s to %s" % (coord, v))
            bored.set_cell_value(coord, v)
            
    # same segment
    try:
        set(Board(), [((4, 4), 5), ((4, 5), 5)], "same segment")
    except IllegalCellValue:
        pass

    # same column
    try:
        c = 8
        set(Board(), [((0, c), 5), ((8, c), 5)], "same column")
    except IllegalCellValue:
        pass

    # same square
    try:
        set(Board(), [((0, 0), 5), ((2, 2), 5)], "same square")
    except IllegalCellValue:
        pass


In [ ]:
def testing_cell_get_guesses():
    print("testing_cell_get_guesses()")

    b = Board()

    b.get_cell(4, 4).set_value(4)
    b.get_cell(4, 5).set_value(5)

    print("cell(4,5) guesses: %s" % (b.get_cell(4, 5).get_guesses()))
    print("cell(4,6) guesses: %s" % (b.get_cell(4, 6).get_guesses()))
    print("cell(6,3) guesses: %s" % (b.get_cell(6, 3).get_guesses()))


In [ ]:
def testing_init_segments():
    print("testing_init_segments()")

    board = Board()
    # board.dump_all_segments()
    
    board.dump_cell(board.get_cell(4,7))
    # board.dump_segment("row", (0, 0))  # TODO 
    board.dump_nine(Square, (0, 0))
    board.dump_nine(Row, 0)
    board.dump_nine(Column, 8)


In [ ]:
import unittest

class BasicInitFundamentalsTestCase(unittest.TestCase):
    def setUp(self):
        self.board = Board()
    
    def test_cells_per_board(self):
        self.assertEqual(len(self.board._all_cells), 81)
        
    def test_cells_per_segment(self):
        segment = self.board._all_row_segments[(1,6)]
        self.assertEqual(len(segment._cells), 3)

    def test_cells_per_nine(self):
        row = self.board._all_rows[3]
        self.assertEqual(len(row._cells), 9)

        column = self.board._all_columns[3]
        self.assertEqual(len(column._cells), 9)

        square = self.board._all_squares[(6, 3)]
        self.assertEqual(len(square._cells), 9)

    def test_segments_per_nine(self):
        pass
    
    def test_segments_per_board(self):
        pass
    
    def test_segment_siblings(self):
        pass
    
    def test_board(self):
        # rows, columns per board
        pass
    
class BasicFunctionalTestCases(unittest.TestCase):
    def test_add_value_twice_to_same_row(self):
        board = Board()
        board.set_cell_value((4,4), 5)
        self.assertRaises(IllegalCellValue, board.set_cell_value, (4,5), 5)

    def test_add_value_to_cell_with_value(self):
        board = Board()
        board.set_cell_value((4,4), 5)
        self.assertRaises(IllegalCellValue, board.set_cell_value, (4,4), 4)

    def test_clear_value_from_empty_cell(self):
        board = Board()
        self.assertRaises(IllegalCellValue, board.clear_cell_value, (4,5))

class ShouldNotHappenTestCases(unittest.TestCase):
    def test_add_bad_value_to_cell(self):
        board = Board()
        self.assertRaises(IllegalCellValue, board.set_cell_value, (4,5), 23)


In [ ]:
def specialized_tests():
    # testing_init_segments()
    # testing_board_set_cell_value()
    # testing_cell_get_guesses()
    # testing_set_cell_value()
    testing_board_set_cell_value2()


In [ ]:
# TODO - should move these into unittest
def manual_tests():
    import random
    print_testing("testing basic structural integrity")

    board = Board()

    CELLS_PER_THREE = 3
    CELLS_PER_NINE = 9
    ROWS_PER_BOARD = 9
    COLUMNS_PER_BOARD = 9
    CELLS_PER_BOARD = COLUMNS_PER_BOARD * ROWS_PER_BOARD

    SEGMENTS_PER_ROW = 3
    SEGMENTS_PER_COLUMN = 3
    SEGMENTS_PER_SQUARE = 6
    SEGMENTS_PER_BOARD = COLUMNS_PER_BOARD * SEGMENTS_PER_COLUMN + ROWS_PER_BOARD * SEGMENTS_PER_ROW

    PARENTS_PER_SEGMENT = 2
    PARENTS_PER_CELL = 5

    print_testing("... board")            
    # board should have:
    # ... 81 cells 
    assert len(board._all_cells) == CELLS_PER_BOARD
    # ... 9 rows and 9 columns
    assert len(board._all_rows) == ROWS_PER_BOARD
    assert len(board._all_columns) == COLUMNS_PER_BOARD
    # ... 27 column segments and 27 row segments

    print_testing("... rows / columns")            
    # row/column nines should have:
    # row = board._all_rows[8]
    row = board._all_rows[random.choice(Board.ROW_NUMBERS)]
    # column = board._all_columns[3]
    column = board._all_columns[random.choice(Board.COLUMN_NUMBERS)]
    print_testing("using", column._id, ",", row._id)
    # ... 3 segments
    assert len(row._all_segments) == SEGMENTS_PER_ROW, len(column._all_segments) == SEGMENTS_PER_COLUMN
    # ... 9 cells
    assert len(row._cells) == CELLS_PER_NINE, len(column._cells) == CELLS_PER_NINE

    print_testing("... squares")            
    # square nines should have:
    square = board._all_squares[(3, 6)]
    # ... 6 segments
    assert len(square._all_segments) == SEGMENTS_PER_SQUARE
    # ... 9 cells
    assert len(square._cells) == CELLS_PER_NINE

    print_testing("... segments")            
    # segments should have:
    # ... 2 parents: square and (row or column)
    # ... 4 siblings: 2 row/column siblings and 2 square siblings
    # ... 3 cells

    print_testing("... cells")            
    # cells should have:
    cell = board._all_cells[5,7]
    # ... 5 parents: 3 nines and 2 segments
    assert len(cell._parents) == 5


In [ ]:
def display_board(board):
    from IPython.display import clear_output, display_html

    clear_output()

    table = '<table style="border-color: black; border-width: 5px;">\n'
    for r in board.ROW_NUMBERS:
        table += '<tr>'
        for c in board.COLUMN_NUMBERS:
            v = board.get_cell(r, c)._value
            table += '<td style="border-style: solid; border-width: thin; height: 20px; width: 25px;">'
            if v is None:
                table += '&nbsp;'
            else:
                table += str(v)
            table += '</td>'
        table += '</tr>\n'
    table += '</table>'
    display_html(table, raw=True)


In [ ]:
def main():
    b = Board()
    b.set_cell_value((4,4), 7)
    b.set_cell_value((1,7), 4)
    display_board(b)


In [ ]:
if __name__ == "__main__":
    testing = False
    
    if not testing:
        main()
    else:
        # specialized tests
        if True:
            specialized_tests()
        
        if True:
            unittest.main(argv=['first-arg-is-ignored'], 
                          exit=False, 
                          verbosity=2)
        
        # basic tests - all should pass
        if True:
            manual_tests()
